# Using GIXStapose to Generate Reproducible Figures

Imagine, you've just finished running a parameter sweep of 10 different temperatures, 10 different pressures, and 10 different compounds. You've used a GUI (perhaps the GIXStapose GUI!) for chemical structure visualization and painstakenly rotated and zoomed to create some really nice figures--it took you a day and a half, but now you're ready to add these figures to your publication. However, when you show the figures you made to your advisor, she asks for a view from a different angle and using a different color scheme...

GIXStapose is designed to save you time in these situations by generating and using metadata for orienting and visualizing chemical structures and their diffraction patterns. In this notebook, we demonstrate how to use the GIXStapose library outside of the GUI, including:
- reproducing the exact figure seen in the GUI 
- exploring real (messy) simulation data

![screenshot.png](../gixstapose/data/screenshot.png)

In [ ]:
import PIL
import matplotlib.pyplot as plt
from fresnel import camera, pathtrace

from gixstapose.draw_scene import get_scene, get_info
from gixstapose.diffractometer import Diffractometer

We'll use the `get_scene` function to load a gsd file into a [`fresnel.Scene`](https://fresnel.readthedocs.io/en/stable/examples/00-Basic-tutorials/04-Scene-properties.html).

`get_scene` by default will use the [cpk color map](https://en.wikipedia.org/wiki/CPK_coloring#Typical_assignments), but you can also use any [matplotlib colormap](https://matplotlib.org/tutorials/colors/colormaps.html). This is especially useful when working with coarse-grained or other non-atomistic systems.

In [ ]:
scene, info = get_scene("../gixstapose/data/sc10.pdb")

Next we can set a `fresnel.camera` object with the same parameters as the one shown above in the GUI and connect that camera object to the scene.

You can easily get the camera parameters from the GUI by selecting "Print Camera" in the GIXStapose menu bar.

In [ ]:
cam = camera.Orthographic(
    position = [1.412, 0.698, 0.720],
    look_at  = [0.000, 0.000, 0.000],
    up       = [-0.292, -0.333, 0.896],
    height   = 1.500
)
scene.camera = cam

The next cell will take a minute to run but shows how to easily recreate the same image as the GUI using fresnel's high quality pathtracer.

In [ ]:
output = pathtrace(scene, light_samples=40, w=600, h=600)
output

Then we can save this output using python image library (PIL).

In [ ]:
image = PIL.Image.fromarray(output[:], mode='RGBA')
image.save("structure.png", dpi=(300, 300))

In a similar fashion, we can use the `load_compound` function to load the compound particle positions and box into the `Diffractometer` class and then choose a diffraction angle based on the camera position using the `diffract_from_camera` function.

In [ ]:
d = Diffractometer(length_scale=1.0)
d.load(info["positions"], info["box"])
d.diffract_from_camera(cam)
fig, ax = d.plot()

And the diffraction pattern can be easily saved using the `savefig` function.

In [ ]:
fig.savefig("diffraction.png")

Finally let's look at a messy example using real simulation data. The original xml file was taken from [https://doi.org/10.18122/cme_lab/4/boisestate](https://doi.org/10.18122/cme_lab/4/boisestate).

The original file can be found in p3ht_data/1000_oligomers/crystallinities/crystalline.xml. The file was converted from an xml to a gsd using [cmeutils](https://github.com/cmelab/cmeutils) [xml_to_gsd](https://github.com/cmelab/cmeutils/blob/8e33350630891cde89c9ba82a9adc682407c320f/cmeutils/gsd_utils.py#L192) function, and reduced to contain only the backbone carbons (type CA) using [snap_delete_types](https://github.com/cmelab/cmeutils/blob/8e33350630891cde89c9ba82a9adc682407c320f/cmeutils/gsd_utils.py#L141) so it would load more quickly. 

This is from one of our united-atom simulations of organic photovoltaic polymers, specifically [P3HT](http://gisaxs.com/index.php/Material:P3HT). Even with these simplifications, it still contains 60,000 particles, so the following cells may be a little slow.

In [ ]:
p3ht_file = "../gixstapose/data/crystalline_CAonly.gsd"

Next let's choose a camera position. I chose this camera position by using the GUI to find distinct diffraction peaks.

In [ ]:
cam = camera.Orthographic(
   position = [2.892, 5.120, -0.830],
   look_at =  [-0.797, 0.604, 0.862],
   up =       [-0.733, 0.390, -0.557],
   height =   170
)

Next let's create a scene from this compound--I'm going to use a custom colormap to set the CA type to be gray. There's a lot of additional modifications that can be done to the fresnel scene--I highly recommend [their tutorials](https://fresnel.readthedocs.io/en/stable/examples/00-Basic-tutorials/00-Introduction.html). But for this example, lets just connect the camera and render it!

In [ ]:
p3ht_scene, info = get_scene(p3ht_file, color={"CA": "gray"}, scale=4)
p3ht_scene.camera = cam
p3ht_output = pathtrace(p3ht_scene, light_samples=40, w=600, h=600)
p3ht_output

At this particular angle we'll draw your attention to parallel stripes of gray separated by black shadowy regions: These stripes are "lamellae" and are a periodic feature that should be measurable with the GIXStapose diffractometer. (Reminder! I used GIXStapose GUI to find an orientation with measurable lamellar peaks and saved its camera orientation so we *could* return to this precise orientation!)

In [ ]:
d = Diffractometer(length_scale=1)
d.load(info["positions"], info["box"])
dp = d.diffract_from_camera(cam)
fig, ax = d.plot()

Aw yeah, just look at that classic lamellar structure! The reflections of the (100) and (200) crystallographic planes are clearly visibile, with a hint of (300)! In this diffraction pattern, the wave vectors pointing to these yellow/green peak centers can be used to derive average lamellar spacings that would be a mess to deduce from the direct visualization. This example showcases how looking at chemical structures through the diffraction lens *first* can identify periodic features that may not be immediately visible to the eye. Once we know a periodic feature exists, we can pull up the chemical structure visualization and look for features at the expected length scales. We can also use this orientation as a reference point to generate a sequence of rotations that reveal other structures (e.g., for you P3HT experts, an orientation that more clearly reveals the spacing of the pi-pi stacking of the P3HT backbones).

Finally, if you want to get the reciprocal space values of the peaks in your diffraction pattern, we provide an interactive class called `PeakLabeller` you can use. Sometimes the interactive backends of matplotlib are a little tricky, so you may need to restart your kernel and reload some information.

In [ ]:
%matplotlib qt5

In [ ]:
# This cell will reload necessary information if needed
import matplotlib.pyplot as plt

from fresnel import camera

from gixstapose.draw_scene import get_info
from gixstapose.diffractometer import Diffractometer, PeakLabeller


p3ht_file = "../gixstapose/data/crystalline_CAonly.gsd"
info = get_info(p3ht_file)

cam = camera.Orthographic(
   position = [2.892, 5.120, -0.830],
   look_at =  [-0.797, 0.604, 0.862],
   up =       [-0.733, 0.390, -0.557],
   height =   170
)

d = Diffractometer(length_scale=1)
d.load(info["positions"], info["box"])
dp = d.diffract_from_camera(cam)

In [ ]:
fig, ax = d.plot()

PeakLabeller(ax)

In [ ]:
peaks = [0.057, 0.290]
for peak in peaks:
    print(f"The peak at {peak} 1/Å corresponds to a periodic spacing of {1/peak:.2f} Å.")

The figure below was created using `PeakLabeller`. We can see that these periodic distances are reasonable for the lamellar spacing (\~17 Å) and pi-stacking (\~4 Å) and agree with [previously reported values](https://doi.org/10.1016/j.orgel.2013.02.028).
![labelled peaks](../gixstapose/data/labelledpeaks.png)